<a href="https://colab.research.google.com/github/robinjacobroy/Odonates/blob/master/YOLO_cutouts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [9]:
%cd /gdrive/My\ Drive

/gdrive/My Drive


In [10]:
%cd YOLO/darknet-master/

/gdrive/My Drive/YOLO/darknet-master


In [0]:
#!ls ../Field3_new/ > field3.txt
%pycat field3.txt

In [12]:
from google.colab import files
from io import BytesIO
uploaded = files.upload()

Saving field3.txt to field3.txt


In [0]:
cp ../Makefile .

In [15]:
!make

chmod +x *.sh
g++ -std=c++11 -Iinclude/ -I3rdparty/stb/include -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
g++ -std=c++11 -Iinclude/ -I3rdparty/stb/include -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/http_stream.cpp -o obj/http_stream.o
./src/http_stream.cpp: In member function ‘bool JSON_sender::write(const char*)’:
./src/http_stream.cpp:228:21: warning: unused variable ‘n’ [-Wunused-variable]
                 int n = _write(client, outputbuf, outlen);
                     ^
gcc -Iinclude/ -I3rdparty/stb/include -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/gemm.c -o obj/gemm.o
./src/gemm.c: 

In [16]:
!./darknet

usage: ./darknet <function>


In [0]:
#darknet.exe detector test cfg/coco.data cfg/yolov3.cfg yolov3.weights -thresh 0.25 -dont_show -save_labels < data/new_train.txt

In [17]:
!./darknet detector test data/odonata.data yolov3-odonata-test.cfg backup/yolov3-odonata_final.weights -thresh 0.25 -dont_show -save_labels < field3.txt

   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Shortcut Layer: 5
   9 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
  10 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
  11 Shortcut Layer: 8
  12 conv    256       3 x 3/ 2    104 x 104 x 128 ->   52 x  52 x 256 1.595 BF
  13 conv    128       1 x 1/ 1     52 x  52 x 256 ->   52 x  5

In [0]:
ls boxed/ | wc -l

0


In [0]:
#code to crop out odonates from YOLO predicitions


import os,cv2,csv,argparse
import numpy as np
from matplotlib import pyplot as plt

    
            
input_dir="../Field3_new/"
out_dir="../Odonates/cropped_field3/"

for fln in sorted(os.listdir(input_dir)):
  if fln.startswith("DSC") and fln.endswith(".JPG"):
    #print(os.path.join(input_dir,fln))
    im=cv2.imread(os.path.join(input_dir,fln))
    #print(im.shape)
    W=im.shape[1]
    H=im.shape[0]
    lst=[]
    try:
      with open(os.path.join(input_dir,fln.replace(".JPG",".txt"))) as cs:
        red=csv.reader(cs,delimiter=' ')
        for itm in red:
          lst=list(map(float,itm))
    except:
      pass
    if not len(lst)==0:
  
      #center coordinates of the bounding box
      x=lst[1]*W
      y=lst[2]*H


      #width and height of the bounding box
      w=lst[3]*W
      h=lst[4]*H
      
      #start and end points of the bounding box
      start_x=int(x-w/2)
      start_y=int(y-h/2)

      end_x=int(x+w/2)
      end_y=int(y+w/2)

      #crop image
      im_crop=im[start_y:end_y,start_x:end_x]
      #cv2.rectangle(im,(start_x,start_y),(end_x,end_y),(0,0,255),thickness=10)
      #plt.imsave(os.path.join(out_dir,fln),im_crop)
      cv2.imwrite(os.path.join(out_dir,fln),im_crop)
      #cv2.imwrite("./results_crop/{}".format(fln),im_crop)
      
    

In [0]:
#code to mask odonates using sobel operation

import os,cv2,csv,argparse
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
           
            
input_dir="./nattakom_field/"
out_dir="./results_sobel/"


for fln in sorted(os.listdir(input_dir)):
  if fln.endswith(".jpg"):
    im=cv2.imread(os.path.join(input_dir,fln))
    img=im.copy()
    W=im.shape[1]
    H=im.shape[0]
    lst=[]
    try:
      with open(os.path.join(input_dir,fln.replace(".jpg",".txt"))) as cs:
        red=csv.reader(cs,delimiter=' ')
        for itm in red:
          lst=list(map(float,itm))
    except:
      pass
    if not len(lst)==0:
  
      #center coordinates of the bounding box
      x=lst[1]*W
      y=lst[2]*H


      #width and height of the bounding box
      w=lst[3]*W
      h=lst[4]*H
      
      #start and end points of the bounding box
      start_x=int(x-w/2)
      start_y=int(y-h/2)

      end_x=int(x+w/2)
      end_y=int(y+w/2)

      #crop image
      im_crop=im[start_y:end_y,start_x:end_x].copy()
      
    
      ddepth = cv2.CV_16S
    
      src = cv2.GaussianBlur(im_crop, (7, 7), 0)
      gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
      # Gradient-X
      #grad_x = cv2.Scharr(gray,ddepth,1,0)
      grad_x = cv2.Sobel(gray, ddepth, 1, 0, ksize=3, scale=1, delta=0, borderType=cv2.BORDER_DEFAULT)
      # Gradient-Y
      grad_y = cv2.Scharr(gray,ddepth,0,1)
      #grad_y = cv2.Sobel(gray, ddepth, 0, 1, ksize=3, scale=1, delta=0, borderType=cv2.BORDER_DEFAULT)
    
      abs_grad_x = cv2.convertScaleAbs(grad_x)
      abs_grad_y = cv2.convertScaleAbs(grad_y)
      grad = cv2.addWeighted(abs_grad_x, 0.5, abs_grad_y, 0.5, 0)
    
      #thresholding the region inside the bounding box
      threshMap = cv2.threshold(grad \
                                , 0, 255,cv2.THRESH_BINARY|cv2.THRESH_OTSU)[1]
      kernel=cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(15,15))
      kernel1=np.ones((15,15),np.uint8)
      kernel2=np.ones((3,3),np.uint8)
      #morphological operations to make the object mask
      dilated=cv2.dilate(threshMap,kernel1,iterations=2)
      closing = cv2.morphologyEx(dilated, cv2.MORPH_CLOSE, kernel1)
      eroded=cv2.erode(closing,kernel2,iterations=3)
      #draw the biggest contour, which could be the object
      _,contours,_=cv2.findContours(eroded,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
      drawing=np.ones(dilated.shape,np.uint8)*255
      drawing=cv2.cvtColor(drawing,cv2.COLOR_GRAY2BGR)
      c=max(contours,key=cv2.contourArea)
      cv2.drawContours(drawing,[c],0,(0,0,0),-1)
      
      final=cv2.bitwise_and(im_crop,drawing)
      final[np.where((final==[0,0,0]).all(axis=2))]= [255,0,255] 
      #output=cv2.hconcat([im_crop,final]) 
      #paste the masked region inside the cutout to the original image
      orig=Image.fromarray(img)
      cropped=Image.fromarray(final)
      orig.paste(cropped,(start_x,start_y))
      cv2.imwrite(os.path.join(out_dir,fln),np.uint8(orig))
      